# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [1]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [2]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [3]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [4]:
# insert these datasets into our db
auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [6]:
# for the tips df/table, get the total number of man and woman
len(tips[tips.sex == 'Female']), len(tips[tips.sex == 'Male'])

(87, 157)

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [ ]:
# in sql 
query = None
c.execute(query).fetchall()

In [31]:
# using pandas, select the average amount tipped for time
tips.groupby(['time']).total_bill.mean()

time
Lunch     17.168676
Dinner    20.797159
Name: total_bill, dtype: float64

In [ ]:
# use sql to recreate this query
query = None
c.execute(query).fetchall()

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [ ]:
# use pandas, get the average total bills for a party larger than 2 for every size of party

# using pandas



In [ ]:
# use sql to recreate this query
query = None
c.execute(query).fetchall()

In [32]:
# your turn - use pandas, get the maximum value of total bill for female non smoker

# using group by?
tips.groupby(['sex', 'smoker']).total_bill.max()['Female', 'No']

35.83

In [7]:
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
# how to get this without using group by?
tips[(tips.sex == 'Female') & (tips.smoker == 'No')].total_bill.max()

35.83

In [15]:
# use sql to recreate this query
query = '''SELECT * 
            FROM Tips 
            '''
pd.DataFrame(c.execute(query).fetchall())

,0,1,2,3,4,5,6,7
0,0,16.99,1.01,Female,No,Sun,Dinner,2
1,1,10.34,1.66,Male,No,Sun,Dinner,3
2,2,21.01,3.50,Male,No,Sun,Dinner,3
3,3,23.68,3.31,Male,No,Sun,Dinner,2
4,4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...,...
239,239,29.03,5.92,Male,No,Sat,Dinner,3
240,240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,242,17.82,1.75,Male,No,Sat,Dinner,2


In [34]:
#### use sql to recreate this query
query = '''SELECT MAX(total_bill) 
            FROM Tips 
            WHERE sex = "Female"
                AND smoker = "No"'''

c.execute(query).fetchall()[0][0]

35.83

In [20]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips
tips.groupby(['size']).tip.mean().sort_values(ascending=False).head(2)

size
6    5.225000
4    4.135405
Name: tip, dtype: float64

In [26]:
# use sql 
query = '''SELECT AVG(tip) a
            FROM Tips 
            GROUP BY size 
            ORDER BY a DESC 
            LIMIT 2'''
c.execute(query).fetchall()

[(5.225,), (4.135405405405407,)]

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [35]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Mike W           1
1    Mitch           4
2   Mike R           0
  students  projects  grades
0   Raizel         1      80
1    Mitch         2      80
2     Eric         1      99


In [36]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Mike W,1,NaN,NaN
1,Mitch,4,2.0,80.0
2,Mike R,0,NaN,NaN


In [44]:
# sql 
query = '''SELECT * FROM small_quiz'''
c.execute(query).fetchall()

[(0, 'Raizel', 1, 80), (1, 'Mitch', 2, 80), (2, 'Eric', 1, 99)]

In [43]:
# sql 
query = '''SELECT * FROM small_grade'''
c.execute(query).fetchall()

[(0, 'Mike W', 1), (1, 'Mitch', 4), (2, 'Mike R', 0)]

In [47]:
[col[0] for col in c.description]

['index', 'students', 'quiz_score', 'index', 'students', 'projects', 'grades']

In [45]:
# sql 
query = '''SELECT * FROM small_grade LEFT JOIN small_quiz ON small_grade.students = small_quiz.students'''
c.execute(query).fetchall()

[(0, 'Mike W', 1, None, None, None, None),
 (1, 'Mitch', 4, 1, 'Mitch', 2, 80),
 (2, 'Mike R', 0, None, None, None, None)]

In [ ]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

In [48]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(1, 'Mitch', 2, 80, 1, 'Mitch', 4)]

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [50]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id 
            FROM `left table` 
            WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col 
            FROM table 
            ORDER BY col 
            LIMIT 1 
            OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""